## Load the libraries

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import descartes as ds
import geopandas as gdp
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.windows import Window
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
from mpl_toolkits.mplot3d import Axes3D
from osgeo import gdal
from matplotlib import cm
from rasterio.enums import Resampling
from rasterio.windows import get_data_window

## Rasterio

### Load the data

In [12]:
with rasterio.Env(GTIFF_FORCE_RGBA=True, CPL_DEBUG=True):
    with rasterio.open('/Users/nicolasdemuynck/BeCode/Github/finalchallenge/DHMVIIDTMRAS1m_k13/GeoTIFF/DHMVIIDTMRAS1m_k13.tif') as img:
        ras_meta = img.profile

### Show image

In [13]:
rasterio.plot.show (img, cmap = 'bone')

RasterioIOError: Dataset is closed: /Users/nicolasdemuynck/BeCode/Github/finalchallenge/DHMVIIDTMRAS1m_k13/GeoTIFF/DHMVIIDTMRAS1m_k13.tif

### Image characteristics

In [14]:
print(img.count) # shows the amount of bands
print(img.bounds) # shows the boundaries of the image
print(img.height) # shows the width
print(img.width) # shows the height
print(img.crs) # shows the coordinate system

1
BoundingBox(left=66000.0, bottom=198000.0, right=98000.0, top=218000.0)
20000
32000
LOCAL_CS["Belge 1972 / Belgian Lambert 72",GEOGCS["Belge 1972",DATUM["unknown",SPHEROID["unretrievable - using WGS84",6378137,298.257223563],TOWGS84[-106.869,52.2978,-103.724,0.3366,-0.457,1.8422,-1.2747]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],AUTHORITY["EPSG","31370"],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]


### Get a window from a raster using coordinates

In [15]:
left = 68750.0
bottom = 210250.0
right = 71500.0
top = 213250.0

rst = img.read(1, window=from_bounds(left, bottom, right, top, img.transform))

RasterioIOError: Dataset is closed: /Users/nicolasdemuynck/BeCode/Github/finalchallenge/DHMVIIDTMRAS1m_k13/GeoTIFF/DHMVIIDTMRAS1m_k13.tif

In [16]:
rasterio.plot.show (rst, cmap = 'terrain')

plt.show ()

NameError: name 'rst' is not defined

### Get a conversion from spherical coordinates to Belgian Lambert 72

In [17]:
# https://epsg.io/map#srs=31370&x=-530060.03&y=-6145691.18&z=21&layer=streets
# http://zoologie.umons.ac.be/tc/algorithms.aspx

### Make window over a specific coordinate (in Belgian Lambert 72)

In [18]:
house_lamb_long =  71398.88
house_lamb_lat = 213899.67

left = house_lamb_long - 100
right = house_lamb_long + 100
bottom = house_lamb_lat - 100
top = house_lamb_lat + 100

housemap = img.read(1, window=from_bounds(left, bottom, right, top, img.transform))

rasterio.plot.show (housemap, cmap = 'terrain')

plt.show ()

RasterioIOError: Dataset is closed: /Users/nicolasdemuynck/BeCode/Github/finalchallenge/DHMVIIDTMRAS1m_k13/GeoTIFF/DHMVIIDTMRAS1m_k13.tif

### Save window as a .tif file

In [19]:
housemap.shape

NameError: name 'housemap' is not defined

In [20]:
with rasterio.open('/Users/nicolasdemuynck/BeCode/Github/finalchallenge/output_window.tif', 'w', **ras_meta) as window:
    window.write(housemap, indexes=1)

NameError: name 'housemap' is not defined

## Make a 3D plot

### Load the new .tif file

In [21]:
tif = gdal.Open('output_window.tif')
house_tif = tif.ReadAsArray ()

In [22]:
print (type(tif))
print (type(house_tif))

<class 'osgeo.gdal.Dataset'>
<class 'numpy.ndarray'>


### Extract the valid x,y,z coordinates

In [108]:
width = tif.RasterXSize
height = tif.RasterYSize
gt = tif.GetGeoTransform ()

In [110]:
x = (width * gt[1]) + gt[0]
y = (height * gt[5]) + gt[3]

In [111]:
X = np.arange(gt[0], x, gt[1])
Y = np.arange(gt[3], y, gt[5])

### Creation of a simple grid without interpolation

In [112]:
X, Y = np.meshgrid(X, Y)

### Plot the raster

In [1]:
fig, axes = plt.subplots(subplot_kw={'projection': '3d'})
axes.plot_surface(X, Y, house_tif, rstride=1, cstride=1, cmap=cm.coolwarm,linewidth=0, antialiased=False)

NameError: name 'plt' is not defined

In [5]:
df = pd.DataFrame(housemap, columns=['Column1', 'Column2'])

NameError: name 'housemap' is not defined

NameError: name 'StructType' is not defined